 # Segmenting and Clustering Neighborhoods in Toronto
 
 Peer-graded Assignment

In [150]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np 
import random 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
%matplotlib inline 

In [151]:
Canada_PC =requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
BSoup=BeautifulSoup(Canada_PC, 'lxml')
tab = str(BSoup.table)
dfs = pd.read_html(tab)
df=dfs[0]
# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Postal Code','Borough'], sort= False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [152]:
print("Shape of the table above:",df2.shape)

Shape of the table above: (103, 3)


#### Lets Use the Geocoder csv file to create the following dataframe with coordinates of the neighborhoods 

In [153]:
Geospatial = pd.read_csv("http://cocl.us/Geospatial_data")
Geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now merging 2 dataframes df2 and Geospatial into New dataframe

In [154]:
Ndf = pd.merge(df2 , Geospatial, on ="Postal Code", how = "outer")
# Change the Postal Code to Postalcode
Ndf.rename(columns ={'Postal Code':'PostalCode'}, inplace = True)
Ndf.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Explore and cluster the neighborhoods in Toronto

In [155]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [156]:
# Sorting
# set index for only Downtown Toronto
Ndf = Ndf[Ndf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
Ndf=Ndf.drop(['PostalCode'], axis=1)
Ndf.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [157]:
CLIENT_ID = 'YAWMB5WRVUZLYS2KD50IRDX4G2TO4PNLHW3XZWIJZS1C0LZP' # your Foursquare ID
CLIENT_SECRET = 'CXGGS3SKSMSLSKONG0ANBQQAO3YW2UQQBKQPVAI4N413ASAL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YAWMB5WRVUZLYS2KD50IRDX4G2TO4PNLHW3XZWIJZS1C0LZP
CLIENT_SECRET:CXGGS3SKSMSLSKONG0ANBQQAO3YW2UQQBKQPVAI4N413ASAL


### Let's get the geographical coordinates of Downtown Toronto

In [158]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","Latitude",Latitude_downtown_toronto, "& " "Longitude" ,Longitude_downtown_toronto)

Downtown Toronto Latitude 43.6563221 & Longitude -79.3809161


In [159]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=100
    LIMIT=30
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [160]:
#Listing Neighbourhood in Downtown Toronto
toronto_venues = getNearbyVenues(names=Ndf['Neighbourhood'],
                                   latitudes=Ndf['Latitude'],
                                   longitudes=Ndf['Longitude']
                                  )
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
1,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
2,"Regent Park, Harbourfront",43.654260,-79.360636,studio b,43.653966,-79.361662,Furniture / Home Store
3,"Regent Park, Harbourfront",43.654260,-79.360636,Henrietta Lane,43.653911,-79.361395,Cocktail Bar
4,"Garden District, Ryerson",43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery


In [161]:
toronto_venues.groupby('Neighbourhood').count() # Number of venues for each neighbourhood.

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
Central Bay Street,3,3,3,3,3,3
Christie,1,1,1,1,1,1
"Commerce Court, Victoria Hotel",16,16,16,16,16,16
"First Canadian Place, Underground city",12,12,12,12,12,12
"Garden District, Ryerson",2,2,2,2,2,2
"Harbourfront East, Union Station, Toronto Islands",2,2,2,2,2,2
"Kensington Market, Chinatown, Grange Park",5,5,5,5,5,5
"Regent Park, Harbourfront",4,4,4,4,4,4


In [162]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(loc=0, column='Neighbourhood', value=toronto_venues['Neighbourhood'])
toronto_onehot.shape

(83, 51)

In [163]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,American Restaurant,Art Gallery,Bakery,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,Burrito Place,...,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.0000,0.000000,0.0,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0
1,Central Bay Street,0.0000,0.0000,0.000000,0.0,0.0000,0.0,0.000000,0.000000,0.0000,...,0.333333,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0
2,Christie,0.0000,0.0000,0.000000,0.0,0.0000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0
3,"Commerce Court, Victoria Hotel",0.0625,0.0625,0.062500,0.0,0.0625,0.0,0.000000,0.000000,0.0625,...,0.062500,0.0625,0.0625,0.0,0.0625,0.0,0.0,0.000000,0.0,0.0
4,"First Canadian Place, Underground city",0.0000,0.0000,0.083333,0.0,0.0000,0.0,0.083333,0.083333,0.0000,...,0.000000,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.083333,0.0,0.0


### 10 Most Common venues in each Neighbourhood

In [164]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [165]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Vegetarian / Vegan Restaurant,College Gym,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
1,Central Bay Street,Pharmacy,Coffee Shop,Sandwich Place,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
2,Christie,Nightclub,Vegetarian / Vegan Restaurant,College Gym,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
3,"Commerce Court, Victoria Hotel",American Restaurant,Sandwich Place,Coffee Shop,Gym,Café,Burrito Place,Deli / Bodega,Pub,Salad Place,Seafood Restaurant
4,"First Canadian Place, Underground city",Gift Shop,Cupcake Shop,Gastropub,Café,Gym / Fitness Center,Salad Place,Burger Joint,Building,Gluten-free Restaurant,Bakery


### Making Clusters for Neighbourhood

In [166]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 3, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [167]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Ndf

# merge toronto_grouped with Ndf to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged = toronto_merged.dropna()
toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Cocktail Bar,Furniture / Home Store,Spa,Breakfast Spot,College Gym,Garden,Food Court,Fast Food Restaurant,Farmers Market,Diner
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4.0,Coffee Shop,Art Gallery,College Gym,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner
3,Downtown Toronto,St. James Town,43.651494,-79.375418,4.0,Japanese Restaurant,Breakfast Spot,Italian Restaurant,Diner,Coffee Shop,Gay Bar,Vegetarian / Vegan Restaurant,Cupcake Shop,Garden,Furniture / Home Store
5,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4.0,Pharmacy,Coffee Shop,Sandwich Place,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
6,Downtown Toronto,Christie,43.669542,-79.422564,3.0,Nightclub,Vegetarian / Vegan Restaurant,College Gym,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega


In [168]:
toronto_merged.dropna()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Cocktail Bar,Furniture / Home Store,Spa,Breakfast Spot,College Gym,Garden,Food Court,Fast Food Restaurant,Farmers Market,Diner
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4.0,Coffee Shop,Art Gallery,College Gym,Gastropub,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner
3,Downtown Toronto,St. James Town,43.651494,-79.375418,4.0,Japanese Restaurant,Breakfast Spot,Italian Restaurant,Diner,Coffee Shop,Gay Bar,Vegetarian / Vegan Restaurant,Cupcake Shop,Garden,Furniture / Home Store
5,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4.0,Pharmacy,Coffee Shop,Sandwich Place,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
6,Downtown Toronto,Christie,43.669542,-79.422564,3.0,Nightclub,Vegetarian / Vegan Restaurant,College Gym,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
7,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,4.0,Vegetarian / Vegan Restaurant,Restaurant,Japanese Restaurant,Garden,Concert Hall,Greek Restaurant,Sushi Restaurant,Taco Place,Tea Room,Food Court
8,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,4.0,Coffee Shop,Gym,College Gym,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega
9,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4.0,Coffee Shop,Restaurant,Gym,Deli / Bodega,College Gym,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market
10,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,4.0,American Restaurant,Sandwich Place,Coffee Shop,Gym,Café,Burrito Place,Deli / Bodega,Pub,Salad Place,Seafood Restaurant
11,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,4.0,College Gym,Gym,Vegetarian / Vegan Restaurant,Garden,Furniture / Home Store,Food Court,Fast Food Restaurant,Farmers Market,Diner,Deli / Bodega


In [ ]:
## enerate maps to visualize your neighborhoods and how they cluster togethe

In [172]:
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters